In [1]:
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch



/Users/tejus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
comments_df = pd.read_csv('../data/YTcomments.csv')
comments_df.head()

,Comment
0,Copy nothing? More like understand nothing.
1,De macho Alfa lomo plateado pelo en pecho a co...
2,Où sont les voitures ? Cest quoi cest guignols...
3,This thing is just a waste of time where is th...
4,WTF did I just watch?


Let the cleaning begin!!
1) get rid of all non-english comments 
2) ill come up with more stuff 

In [5]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

In [9]:
# Function to detect if a comment is in English
def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False  # Handle cases where detection fails (e.g., empty strings)

# Ensure all entries in "Comment" are strings
comments_df['Comment'] = comments_df['Comment'].astype(str)

# Filter rows where "Comment" is detected as English
en_comments = comments_df[comments_df['Comment'].apply(is_english)]


The dataframe is composed of only english comments, now time to perform the sentiment analysis

Need a model familiar with sarcasm, and one pretrained on social media language